In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
            for cell in data['cells']:
                lines = cell['source']
                if type(lines) == str:
                    lines = lines.split('\n')
                line0  = (lines or [''])[0].strip()
                haskey = lambda key: line0 == f'### {key} ###'
                if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                    code = '\n'.join(lines)
                    module.write(code)
                    module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

!cat /proc/meminfo | grep Mem

ipynb = '!!!'

# %load_ext tensorboard

# !rm -rf "./tensorboard/"


'!!!.ipynb'    ttt-tactics	        ttt-tactics-test.ipynb
 __pycache__   ttt-tactics.ipynb        ttt-tactics-train.ipynb
 tmp	       ttt-tactics-play.ipynb
 ttt-3.ipynb   ttt_tactics.py
INFO:tensorflow:Using local port 23044
INFO:tensorflow:Using local port 19503
INFO:tensorflow:Using local port 19375
INFO:tensorflow:Using local port 20045
INFO:tensorflow:Using local port 19722
INFO:tensorflow:Using local port 17961
INFO:tensorflow:Using local port 18396
INFO:tensorflow:Using local port 24990
INFO:tensorflow:Using local port 16710
INFO:tensorflow:Using local port 21414
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
cuda: False
tensorflow: 2.4.0
python: 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]
module:  ttt_tactics
DIR = ./ttt-tactics
<function savesamples at 0x7fb3440ee550>
<function loadsamples at 0x7fb3440ee790>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7fb3481613a0>
MemTotal:       16393932 kB
MemFree:         1069932 kB
MemAv

In [2]:
import requests
from urllib.request import urlretrieve
from urllib.parse   import quote, urlencode 
from bs4            import BeautifulSoup
from selenium       import webdriver

def get_static_html(url):
    headers  = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    return response.text

def get_dynamic_html(url):
    with webdriver.Chrome() as driver:
        driver.get(url)
        html = driver.page_source
    return html

params = urlencode({
    'searchValue' : 'Кабель АВБбШв 4*70',
    'sidx'        : 'price',
    'sord'        : 'asc',
    'goodsOnPage' : 10,
    'page'        : 1,
    'cst'         : 0,
    'dst'         : '',
    'spec'        : ''}) 
url = 'https://www.etm.ru/catalog/?' + params
print(url)

html = get_static_html(url)
# html = get_dynamic_html(url)

soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())



https://www.etm.ru/catalog/?searchValue=%D0%9A%D0%B0%D0%B1%D0%B5%D0%BB%D1%8C+%D0%90%D0%92%D0%91%D0%B1%D0%A8%D0%B2+4%2A70&sidx=price&sord=asc&goodsOnPage=10&page=1&cst=0&dst=&spec=


In [12]:

tags = list(soup.find_all('div', class_='catalog-col'))
print(len(tags), 'goods on page')
# print(tags[0].prettify())

def parse_good(tag):
    code  = tag.find('span',string='Код товара').find_next('span')
    code  = code.text   
    price = tag.find('span',class_='price')
    price = float(price.text)
    units = tag.find('span',class_='price_by_once')
    found = re.findall(r'\d*\.\d+|\d+', units.text) 
    assert(len(found) == 1)
    units = float(found[0])
    return { 'code': code, 'price': price, 'units': units, 'ppu': price/units }

def parse_goods(tags, loglevel=-1): 
    goods = DataFrame(columns=['code', 'price', 'units', 'ppu'])
    for tag in tags:
        try:
            good  = parse_good(tag)
            goods = goods.append(good,ignore_index=True)
        except Exception as e:
            if loglevel >= 0: traceback.print_exc()
            if loglevel >  0: print(tag.prettify())
    return goods

goods = parse_goods(tags, loglevel=-1) 
goods = goods.sort_values('ppu')
goods

20 goods on page


,code,price,units,ppu
6,2503628,4732.48,16.0,295.78
5,2503628,5915.60,20.0,295.78
1,6003596,338.69,1.0,338.69
9,2990091,384.12,1.0,384.12
0,8727286,396.44,1.0,396.44
7,1731538,419.18,1.0,419.18
8,3213457,419.18,1.0,419.18
2,6498039,428.66,1.0,428.66
10,7020315,432.52,1.0,432.52
4,5117228,440.65,1.0,440.65


In [ ]:
# soup = bs4.BeautifulSoup(html)

# # Find all with a specific attribute

# tags = soup.find_all(src=True)
# tags = soup.select("[src]")

# # Find all meta with either name or http-equiv attribute.

# soup.select("meta[name],meta[http-equiv]")

# # find any tags with any name or source attribute.

# soup.select("[name], [src]")

# # find first/any script with a src attribute.

# tag = soup.find('script', src=True)
# tag = soup.select_one("script[src]")

# # find all tags with a name attribute beginning with foo
# # or any src beginning with /path
# soup.select("[name^=foo], [src^=/path]")

# # find all tags with a name attribute that contains foo
# # or any src containing with whatever
# soup.select("[name*=foo], [src*=whatever]")

# # find all tags with a name attribute that endwith foo
# # or any src that ends with  whatever
# soup.select("[name$=foo], [src$=whatever]")
# You can also use regular expressions with find or find_all:

# import re
# # starting with
# soup.find_all("script", src=re.compile("^whatever"))
# # contains
# soup.find_all("script", src=re.compile("whatever"))
# # ends with 
# soup.find_all("script", src=re.compile("whatever$"))

In [3]:
### TEST COMPOSABLES ###

@composable
def f1(x):
    print('f1',x)
    return x+1

@composable
def f2(x):
    print('f2',x)
    return 1,2.0

@composable
def f3(x,y):
    print('f3',x,y)
    return x,y

@composable
def f3opt(x,y):
    print('f3opt',x,y)
    return (x,y),{'opt':'joder'}

@composable
def f4(x,y,opt=None):
    print('f4',x,y,opt)    

f = f4 << f3 << f2 << f1
f(1)
f = f1 >> f2 >> f3 >> f4
f(1)
f = f1 >> f2 >> f3opt >> f4
f(2)

@composable
def isodd(x): 
    return x%2 != 0

@composable
def iseven(x): 
    return x%2 == 0

@composable
def ispos(x): 
    return x > 0

# p =   ispos & iseven
p =  ~ispos | iseven
pprint(p)
for i in range(20):
    x = i - 10
    if p(x):
        print(x)

f1 1
f2 2
f3 1 2.0
f4 1 2.0 None
f1 1
f2 2
f3 1 2.0
f4 1 2.0 None
f1 2
f2 3
f3opt 1 2.0
f4 1 2.0 joder
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
2
4
6
8


In [ ]:
@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)
    
@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)
    
@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)
        
fun(1)
fun([1,2,3])